In [1]:
import csv
import os
import tensorflow as tf
import keras
from keras import layers
import sklearn.model_selection
import numpy as np
import math

tf.config.set_visible_devices([], 'GPU')


In [2]:
u = csv.DictReader(open('u.csv', encoding='utf8'))
# print(u.fieldnames)

xs = []
for x in u:
    a = eval(x['Preferred Activities'])
    a = list(map(lambda x: x.strip(), a))
    d = eval(x['Bucket list destinations Sri Lanka'])
    d = list(map(lambda x: x.strip(), d))
    # print(a, d)
    xs.append((a, d))

# xs = xs[:10]

ks = list(set([x for a, d in xs for x in a]))
ks.sort()
vs = list(set([x for a, d in xs for x in d]))
vs.sort()
print(len(xs), len(ks), len(vs))


10000 68 157


In [3]:
p = csv.DictReader(open('p.csv', encoding='utf8'))
# print(u.fieldnames)

ps = {}
for x in p:
    n = x['name'].strip()
    if not (x['rating'] and x['user_ratings_total']):
        continue
    r = float(x['rating'])
    t = int(x['user_ratings_total'])
    t = math.log10(t)
    ps[n] = math.log(r*t)

ar = sum(ps.values())/len(ps)
print(len(ps), ar, min(ps.values()), max(ps.values()))

# print(vs)
# print(ps)
psx = np.array([ps[v] if v in ps else ar for v in vs])+1
psx = psx/max(psx)
# print(psx,max(psx))


343 2.439213673429212 0.5899450208665175 3.056359854247776


In [4]:
model = keras.Sequential()
model.add(layers.InputLayer(input_shape=(len(ks),)))
model.add(layers.Dense(units=len(vs), use_bias=False, kernel_initializer='zeros'))
# model.add(layers.Dense(units=128))
# model.add(layers.Dense(units=128))
model.add(layers.Lambda(lambda x: x*psx))
# model.add(layers.Reshape(target_shape=(len(ks), 1)))
# model.add()

#model.compile(optimizer='adam', loss='mse')
#model.fit(gen(traind, 0), steps_per_epoch=len(traind), epochs=1000, callbacks=[tensorboard_callback])

model.summary()


d:\install\envs\ml_workshop\lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 157)            │        10,676 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 157)            │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,676 (41.70 KB)

 Trainable params: 10,676 (41.70 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
#model.set_weights([np.array([[0, 0, 0, 0, 0],[0, 0, 0, 0, 0,],[1, 0, 0, 0, 0]])])

In [6]:
ws = model.get_weights()[0]
ws[:, :] = 0
for a, d in xs:
    for aa in a:
        for dd in d:
            # print(a,aa,dd,ks.index(aa),vs.index(dd))
            ws[ks.index(aa)][vs.index(dd)] += 1

ws = sklearn.preprocessing.minmax_scale(ws, axis=1)

model.set_weights([ws])
print(model.get_weights())


[array([[0.00429185, 0.01287554, 0.02575107, ..., 0.05150215, 0.07725322,
        0.03433476],
       [0.02427185, 0.01456311, 0.02912621, ..., 0.05825243, 0.9223301 ,
        0.00970874],
       [0.00403226, 0.01612903, 0.01209677, ..., 0.03629032, 0.08870968,
        0.03225806],
       ...,
       [0.01578948, 0.01052632, 0.04736842, ..., 0.931579  , 1.        ,
        0.03157895],
       [0.02033898, 0.01016949, 0.02372881, ..., 0.04067796, 0.10508475,
        0.0440678 ],
       [0.02583026, 0.01107011, 0.01845019, ..., 0.05166052, 0.12546125,
        0.02214022]], dtype=float32)]


In [7]:
def f(i):
    # print(i)
    i = [1 if k in i else 0 for k in ks]
    o = model.call(inputs=tf.convert_to_tensor([i]))
    o = tf.math.top_k(o, k=5)
    ps = o.values.numpy().tolist()[0]
    # print(vs)
    o = o.indices.numpy().tolist()[0]
    o = [vs[i] for i in o]
    # print(o)
    return o, ps


def g(z):
    a, b, c = z
    abc, ps = f([a, b, c])
    # return abc
    for x in [c, b, a]:
        x, p = f([x])
        x, p = x[0], p[0]
        if x not in abc:
            abc.insert(0, x)
            ps.insert(0, p)
    return abc[:5], ps[:5]


In [8]:
def t(x):
    if len(x) == 1:
        z = f(x)
    else:
        z = g(x)
    print(','.join(x), dict(zip(z[0], map(lambda x: round(x, 2), z[1]))))


t(['hot springs'])
t(['elephant rides'])
t(['museum visits'])
t(['waterfalls'])
t(['hot springs', 'elephant rides', 'waterfalls'])
t(['cycling', 'historical monuments', 'village homestays'])


hot springs {'Madunagala Hot Water Spring': 0.89, 'Kanniya Hot Springs': 0.83, 'Mahapelessa Hot Springs': 0.8, 'Maha Oya Hot Water Springs': 0.78, 'Sigiriya': 0.16}
elephant rides {'Udawalawe': 0.85, 'Pinnawala': 0.84, 'Sigiriya': 0.2, 'Mirissa Beach': 0.17, 'Trincomalee': 0.15}
museum visits {'Colombo National Museum': 0.83, 'Ratnapura Gem Museum': 0.82, 'Kandy National Museum': 0.81, 'National Museum Galle': 0.81, 'Folk Museum': 0.76}
waterfalls {'Diyaluma Falls': 0.89, "St Clair's Falls": 0.85, 'Bambarakiri Ella': 0.84, 'Bambarakanda Falls': 0.81, 'Bopath Falls': 0.81}
hot springs,elephant rides,waterfalls {'Madunagala Hot Water Spring': 0.96, 'Diyaluma Falls': 0.93, 'Udawalawe': 0.91, 'Pinnawala': 0.89, 'Kanniya Hot Springs': 0.88}
cycling,historical monuments,village homestays {'Ella': 1.71, 'Anuradhapura': 1.1, 'Hatton': 0.97, 'Haputale': 0.96, 'Polonnaruwa': 0.88}


In [10]:
import tqdm

zs = {}
for a, d in tqdm.tqdm(xs[:2]):
    v = len(set(g(a)[0]).intersection(set(d)))
    if v not in zs:
        zs[v] = 0
    zs[v] += 1


  0%|          | 0/10000 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [01:47<00:00, 93.07it/s]


In [11]:
zs

{5: 1, 3: 1}

In [12]:
sum([(k)*v for k, v in zs.items()])/sum(zs.values())/5*100

80.0